# MSOA level segmentation of London to aid decision making in choosing an ideal location for a new restaurant.


## Todo
1. Remove unnecessary columns
2. <strike>Add wordCloud cluster analysis....</strike>
3. <strike>Add new tooltip template</strike>
    1. All relevant stats, including clusters
1. <strike>Analyse optimal cluster </strike>
4. Write narrative
5. Revise mips

## Introduction

## Background


## Data


### Used datasets

## Applied techniques

## Results and discussion

## Conclusion

# Installation and importing of the used python libraries

## Installation

In [1]:
!pip install folium
!pip install wordcloud
!pip install plotly
!pip install xlrd
!pip install bs4
!pip install openpyxl
!pip install matplotlib
!pip install pandas
!pip install wordcloud
!pip install numpy
!pip install ipywidgets
!pip install pyclustertend

You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


...

You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


## Importing the libraries

In [2]:
import folium
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
import requests
import plotly
from wordcloud import WordCloud, STOPWORDS
from bs4 import BeautifulSoup
import io
from zipfile import ZipFile
import time
from ipywidgets import widgets
from IPython.display import display
import re
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from pyclustertend import hopkins
from sklearn.preprocessing import scale

# Constants, configuration and initialisation

In [3]:
LONDON_COORDS = [51.5074, 0.1278]

#### setting default figure size for our convenience

rcParams["figure.figsize"] = 20, 10

#### changing style of matplotlib

mpl.style.use('ggplot')

#### dictionary for the urls
urls = dict()

#### dictionary for the dataframes

dfs = dict()
maps = dict()

## Data sources (URLs)

In [4]:
urls["MSOA_ETH_2011"] = "https://data.parliament.uk/resources/constituencystatistics/Ethnic-group.xlsx"

urls["MSOA_INCOME_2018"] = "https://www.ons.gov.uk/file?uri=%2femploymentandlabourmarket%2fpeopleinwork%2fearningsandworkinghours%2fdatasets%2fsmallareaincomeestimatesformiddlelayersuperoutputareasenglandandwales%2ffinancialyearending2018/totalannualincome2018.csv"

urls["MSOA_TO_LSOA_MAPPING"] = "https://opendata.arcgis.com/datasets/fe6c55f0924b4734adf1cf7104a0173e_0.csv"

urls["MSOA_POP_WEIGHTED_COORDS"] = "https://opendata.arcgis.com/datasets/b0a6d8a3dc5d4718b3fd62c548d60f81_0.geojson"
urls["LSOA_AVERAGE_RENTS"] = "https://data.london.gov.uk/download/average-private-rents-borough/73b9fb07-b5bb-4a53-88b7-c17269879a08/voa-average-rent-borough.xls"

urls["MSOA_GSON"] = "https://opendata.arcgis.com/datasets/826dc85fb600440889480f4d9dbb1a24_1.geojson"

# Data acquisition

## Obtaining ethnical data of UK

In [5]:
dfs = dict()
dfs["MSOA_ETH_2011"] = pd.read_excel(urls["MSOA_ETH_2011"], sheet_name = "Neighbourhood data")
dfs["MSOA_ETH_2011"].head(3)

,ONSConstID,ConstituencyName,MSOAID,MSOAName,DateThisUpdate,DateOfDataset,PopWhiteNbhdNum,PopMixedNbhdNum,PopAsianNbhdNum,PopBlackNbhdNum,PopOtherNbhdNum,PopWhiteNbhd%,PopMixedNbhd%,PopAsianNbhd%,PopBlackNbhd%,PopOtherNbhd%
0,E14000530,Aldershot,E02004753,"Blackwater, Frogmore & Minley",2020-11-12,2011,9368,161,402,137,47,0.926149,0.015917,0.039743,0.013544,0.004647
1,E14000530,Aldershot,E02004802,Hawley Lane & Fox Lane,2020-11-12,2011,5709,141,887,77,40,0.832944,0.020572,0.129413,0.011234,0.005836
2,E14000530,Aldershot,E02004803,Mayfield,2020-11-12,2011,5603,229,1499,153,68,0.741923,0.030323,0.198490,0.020260,0.009004


In [6]:
dfs["MSOA_ETH_2011"]["total_population"] = dfs["MSOA_ETH_2011"][["PopWhiteNbhdNum", "PopMixedNbhdNum", "PopAsianNbhdNum", "PopBlackNbhdNum", "PopOtherNbhdNum"]].sum(axis = 1)

### Filtering the Dataframe to relevant features

In [7]:
dfs["MSOA_ETH_2011_mod"] = dfs["MSOA_ETH_2011"][["MSOAID","total_population",
                                                 "PopAsianNbhd%"]]
dfs["MSOA_ETH_2011_mod"].rename(columns = {'MSOAID': "msoa11cd",
                                       "PopAsianNbhd%": "asian_population_percentage"
                                       }, inplace = True)

C:\Program Files\Python38\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
dfs["MSOA_ETH_2011_mod"].head(3)

,msoa11cd,total_population,asian_population_percentage
0,E02004753,10115,0.039743
1,E02004802,6854,0.129413
2,E02004803,7552,0.198490


## Obtaining income data of UK
This proved to be a bit trickier since by default, non-browser requests are blocked from the source domain. Therefore, the requests library was used.

In [9]:
faking_browser = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}
req = requests.get(urls["MSOA_INCOME_2018"], headers = faking_browser)
io_buff = io.BytesIO(req.content)
dfs["MSOA_INCOME_2018"] = pd.read_csv(io_buff, skiprows = 4, encoding= "ISO8859-2")
dfs["MSOA_INCOME_2018"].head(3)

,MSOA code,MSOA name,Local authority code,Local authority name,Region code,Region name,Total annual income (Ł),Upper confidence limit (Ł),Lower confidence limit (Ł),Confidence interval (Ł)
0,E02004297,County Durham 001,E06000047,County Durham,E12000001,North East,"39,800","47,800","33,100","14,700"
1,E02004290,County Durham 002,E06000047,County Durham,E12000001,North East,"42,200","50,700","35,200","15,500"
2,E02004298,County Durham 003,E06000047,County Durham,E12000001,North East,"40,100","48,000","33,400","14,600"


### Filtering the Dataframe to relevant features
Additionally, the result set will be limited to MSOA ID's whose parent region is London.

In [10]:
dfs["MSOA_INCOME_2018_mod"] =\
 dfs["MSOA_INCOME_2018"][dfs["MSOA_INCOME_2018"]["Region name"].str.match("London")]
dfs["MSOA_INCOME_2018_mod"] = dfs["MSOA_INCOME_2018_mod"]\
[["MSOA code", "Region name", "Total annual income (Ł)"]]

In [11]:
dfs["MSOA_INCOME_2018_mod"]["Total annual income (Ł)"].dtypes

dtype('O')

In [12]:
#### Fixing income formatting related value errors
dfs["MSOA_INCOME_2018_mod"]["Total annual income (Ł)"] = dfs["MSOA_INCOME_2018_mod"]["Total annual income (Ł)"].str.replace(",", "").map(int)

In [13]:
dfs["MSOA_INCOME_2018_mod"].rename(columns = {"MSOA code": "msoa11cd", "Region name": "region_name", "Total annual income (Ł)": "total_annual_income"}, inplace = True)

In [14]:
dfs["MSOA_INCOME_2018_mod"].head(3)

,msoa11cd,region_name,total_annual_income
4000,E02000166,London,52900
4001,E02000167,London,68200
4002,E02000168,London,57000


### Obtaining average rent data

In [ ]:
raw_data = requests.get(urls['LSOA_AVERAGE_RENTS'], headers = faking_browser).content

In [ ]:
io_buff = io.BytesIO(raw_data)
excel_file = pd.ExcelFile(io_buff)
excel_file.sheet_names

In [ ]:
dfs['LSOA_AVERAGE_RENTS'] = excel_file.parse('Raw data', skiprows = 2)

In [ ]:
dfs['LSOA_AVERAGE_RENTS'].head(3)

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'] = dfs['LSOA_AVERAGE_RENTS'][dfs['LSOA_AVERAGE_RENTS']["Year"] == 2019]

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD']["Category"].unique()

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'].groupby(by="Category").count()

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'] = dfs['LSOA_AVERAGE_RENTS_MOD'][["Code", "Category","Average"]][dfs['LSOA_AVERAGE_RENTS_MOD']["Category"] == "All categories"]

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'].head()

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'].drop(columns = ["Category"], inplace = True)
dfs['LSOA_AVERAGE_RENTS_MOD'].columns = ["lad17cd", "average_rent"]

### Obtaining output area code mappings

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING"] = pd.read_csv(urls['MSOA_TO_LSOA_MAPPING'])

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING"].head(3)

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING_MOD"] = dfs["MSOA_TO_LSOA_MAPPING"][["LAD17CD", "MSOA11CD"]]

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING_MOD"].shape

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING_MOD"] = dfs["MSOA_TO_LSOA_MAPPING_MOD"].drop_duplicates()

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING_MOD"].shape

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING_MOD"].rename(columns = {"LAD17CD": "LAD17CD".lower(), "MSOA11CD": "MSOA11CD".lower()}, inplace = True)

In [ ]:
dfs["MSOA_TO_LSOA_MAPPING_MOD"].head()

### Merging "average rent" and "msoa to lsoa mapping" dataframes

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'] = dfs['LSOA_AVERAGE_RENTS_MOD'].merge(dfs["MSOA_TO_LSOA_MAPPING_MOD"], how = "left", left_on = "lad17cd", right_on = "lad17cd")

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'].head(3)

In [ ]:
dfs['LSOA_AVERAGE_RENTS_MOD'].shape

## Obtaining the weighted centroid locations of the MSOAs in the UK
Since the found dataset is provided in JSON format, this acquisition process was a bit different. Pandas' json_normlaize was used to facilitate data processing.

In [ ]:
dfs["MSOA_POP_WEIGHTED_COORDS"] = json_normalize(requests.get(urls["MSOA_POP_WEIGHTED_COORDS"]).json()["features"])

### Filtering the dataframe to relevant features

In [ ]:
dfs["MSOA_POP_WEIGHTED_COORDS_mod"] = dfs["MSOA_POP_WEIGHTED_COORDS"].drop(columns = ["type"])
dfs["MSOA_POP_WEIGHTED_COORDS_mod"].columns = list(c.split(".")[1] for c in dfs["MSOA_POP_WEIGHTED_COORDS_mod"].columns)
dfs["MSOA_POP_WEIGHTED_COORDS_mod"].head(3)

## Merging the obtained DataFrames

In [ ]:
dfs["all"] = dfs["MSOA_INCOME_2018_mod"].merge(
    dfs["MSOA_POP_WEIGHTED_COORDS_mod"], how = "left", left_on = "msoa11cd", right_on = "msoa11cd").merge(
    dfs["MSOA_ETH_2011_mod"], how = "left", left_on = "msoa11cd", right_on = "msoa11cd").merge(
    dfs['LSOA_AVERAGE_RENTS_MOD'], how = "left",  left_on = "msoa11cd", right_on = "msoa11cd"
)

In [ ]:
dfs["all"].head(3)

In [ ]:
dfs["all"].columns.values

In [ ]:
#### A bit of tidying up...removing duplicate columns
dfs["all"].drop(columns = ["objectid", "type"], inplace = True)

In [ ]:
#### Checking for join fails -> counting null values for each column
dfs["all"].isna().sum()

### Housekeeping: deleting dataframes that are no longer used to reduce the notebook's size.

In [ ]:
all_keys = list(dfs.keys())
all_keys

In [ ]:
for k in all_keys:
    if(k!="all"):
        del dfs[k]
del io_buff
del raw_data

In [ ]:
dfs.keys()

## Obtaining geographical data for Chloropeth visualisation

In [ ]:
#### Checking the Geo Json file's structure to find the key to be matched on!
json_response = requests.get(urls["MSOA_GSON"]).json()
json_response.keys()

In [ ]:
json_response["features"][0]["properties"]["msoa11cd"]

### Filtering the retrieved Geo JSON to locations within London

In [ ]:
def add_msoa_markers(passed_map, dfs_all, key_attribute_col):
  for i in dfs_all.index:
    row = dfs_all.loc[i]
    loc = row["coordinates"]
    msoa11nn = row["msoa11nm"]
    msoa11cd = row["msoa11cd"]
    key_attribute_value = row[key_attribute_col] 
    pop = "{} ({})\n{}: {}".format(msoa11nn, msoa11cd, key_attribute_col, key_attribute_value)
    folium.Marker([loc[1], loc[0]],
                        radius = 5,
                        popup= pop                   
                        ).add_to(passed_map)


In [ ]:
isinstance(dfs["all"].loc[0].to_dict()["asian_hospitality_percentage"], np.float)

In [ ]:
def filter_geo_json_with_tooltip(json_response_backup, unique_msoa_codes, dfs_all = None, dfs_tooltip_cols = None, tooltip_name = None):
    
    filtered_list = list()
    
    for row in json_response["features"]:
        current_msoa11cd = row["properties"]["msoa11cd"]
        
        if current_msoa11cd in unique_msoa_codes:
            tooltip_temp = "MSOA ID: {} MSOA name:{}<br>"\
            .format(row["properties"]["msoa11cd"], row["properties"]["msoa11nm"])
            
            if((dfs_all is not None) and (dfs_tooltip_cols is not None)):
                if not isinstance(dfs_tooltip_cols, list):
                    dfs_tooltip_cols = [dfs_tooltip_cols]
                boolean_mask = dfs_all["msoa11cd"] == current_msoa11cd
                row_dict = dfs_all[boolean_mask].to_dict(orient = "list")
                for dfs_tooltip_col in dfs_tooltip_cols:
                    dfs_tooltip_val = row_dict[dfs_tooltip_col][0]
                    float_formatter = "{:.5f}" if isinstance(dfs_tooltip_val, np.float) else "{}"
                    to_be_formatted = "{}: " + float_formatter +"<br>"
                    tooltip_temp += to_be_formatted.format(dfs_tooltip_col, dfs_tooltip_val)
                if tooltip_name is not None:
                    row["properties"][tooltip_name] = tooltip_temp
                else:
                    row["properties"][dfs_tooltip_col[0]] = tooltip_temp
            else:
                row["properties"]["tooltip"] = tooltip_temp
            filtered_list.append(row)
            
    print(len(filtered_list))
    return filtered_list

In [ ]:
MSOA_codes = dfs["all"]["msoa11cd"].unique()
filtered_list  = filter_geo_json_with_tooltip(json_response, MSOA_codes, dfs_all = dfs["all"], dfs_tooltip_cols = 'total_annual_income')
json_response["features"] = filtered_list

In [ ]:
#### Reassigning the features key value so that metadata does not have to be redefined.
json_response["features"]= filtered_list

In [ ]:
json_response["features"][0]["properties"]

## Plotting the Income by MSOAs

In [ ]:
ax = dfs["all"].sort_values(by='total_annual_income', ascending = False)\
.head(30)\
.plot(x ='msoa11nm',
      y = 'total_annual_income',
      kind = "bar",
      label = "Household income")
ax.set_title("The top 30 MSOA areas based on Total annual income")
ax.set_xlabel("MSOA")
ax.set_ylabel("Total income")

### Chloropeth map of Total income

In [ ]:
maps["base_map"] = lambda : folium.Map(LONDON_COORDS, zoom_start = 10)

In [ ]:
maps["income_map"] = maps["base_map"]()
maps["income_choro_layer"] = folium.Choropleth(
    data = dfs["all"],
    columns = ["msoa11cd", 'total_annual_income'],
    geo_data = json_response,
    key_on = "feature.properties.msoa11cd",
    fill_color = "YlOrRd"
)
maps["income_choro_layer"].add_to(maps["income_map"])

maps["gson_popup"] = folium.features.GeoJsonTooltip(fields = ["total_annual_income"], labels = False)
maps["income_choro_layer"].geojson.add_child(maps["gson_popup"])

In [ ]:
#maps["income_map"]

## Plotting the MSOA's with the highest percentage of asian population

In [ ]:
ax = dfs["all"]\
.sort_values(by = "asian_population_percentage",
             ascending = False)\
             .head(30)\
             .plot(x = "msoa11nm", y = "asian_population_percentage", kind = "bar", label = "Asian percentage")
ax.set_ylabel("Ethnicity percentage")
ax.set_xlabel("MSOA")
ax.set_title("Top 30 MSOA based on the percentage of asian population")

### Chloropeth map of asian population's distribution

In [ ]:
MSOA_codes = dfs["all"]["msoa11cd"].unique()
filtered_list  = filter_geo_json_with_tooltip(json_response, MSOA_codes, dfs_all = dfs["all"], dfs_tooltip_cols = "asian_population_percentage")
json_response["features"] = filtered_list

In [ ]:
maps["selected_eth_choro_layer"] = folium.Choropleth(
    data = dfs["all"],
    columns = ["msoa11cd", "asian_population_percentage"],
    geo_data = json_response,
    key_on = "feature.properties.msoa11cd",
    fill_color = "YlOrRd"
)
maps["selected_eth_choro_layer"].geojson.add_child(
    folium.features.GeoJsonTooltip(fields = ["asian_population_percentage"], labels = False)
)

In [ ]:
m2 = maps['base_map']()
m2 = m2.add_child(maps["selected_eth_choro_layer"])
#m2

## FourSquare Data Acquisition

### FourSquare helper functions

In [ ]:
#### Requesting data for one specific location on the map.

def explore_area_nearby(client_id, client_secret, access_token, lat, lng, radius = 500, limit = 100):
    version = '20180605' # Foursquare API version
    base_url_template = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&v={}&radius={}&limit={}'
    lat_lng_suffix = "&ll={},{}"
    base_url = base_url_template.format(client_id, client_secret, access_token, version, radius, limit)
    request_url = base_url + lat_lng_suffix.format(lat, lng)
    #print(request_url)
    resp = requests.get(request_url)
    return resp.json()

#### Helper wrapper function to be used for the function above ("explore_area_nearby")

def get_fs_for_latlng(data, raw = False):
    lat_lng = data
    client_id = "OCJNVVQEBW4JMRT1XQWOMUA4D3BSVJ1ABPSFVWIGNQZ4MHKT"
    client_secret = "4XAPAWARYQL5Y0OLDVCABVNFTC5SXUUH4Q0YYVVHSHNJJ44F"
    access_token = "N2LGNKF1ZJBDYCFOYITCMFEUSMFYEXSZWEOEG2TNH15H1GEC"
    radius = 1000
    limit = 500
    raw_response = explore_area_nearby(client_id, client_secret, access_token, lat_lng[1], lat_lng[0], radius, limit)
    if(raw):
        return raw_response
    
    return raw_response["response"]["groups"][0]["items"]

#### Helper functions to extract relevant fields from the JSON response of FourSquare's API

def get_venue_category(i, raw_response):
    return raw_response["response"]["groups"][0]["items"][i]["venue"]["categories"][0]["name"]

    
def get_venue_type(i, raw_response):
    return raw_response["response"]["groups"][0]["items"][i]["reasons"]["items"][0]["type"]

def get_venue_name(i, raw_response):
    return raw_response["response"]["groups"][0]["items"][i]["venue"]["name"]

def get_venue_lat_lng(i, raw_response):
        return [raw_response["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"], 
                raw_response["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]]

#### Placing the above functions into a dictionary for programming convenience
venue_extractor_functions = dict(venue_name = get_venue_name, 
                                 venue_location = get_venue_lat_lng,
                                 venue_type =  get_venue_type, 
                                 venue_category = get_venue_category)

#### Function to be used in a loop to extract each selected relevant field with one of the above functions

def get_venue_fields(raw_response, venue_field_extractor_function):
    extracted_fields = []
    for i in range(0, len(raw_response["response"]["groups"][0]["items"])):
        extracted_fields.append(venue_field_extractor_function(i, raw_response))
    return extracted_fields

#### Since FourSquare's free tier only allows for 500 requests within an hour 
#### and rejects calls with the same API credentials in the next 2 hours,
#### the helper function below aims to automatize this waiting process

def patient_foursquare_calls(dfs):
    #### Venue dataframe template
    venue_ds_template = dict(venue_name = "", venue_location = "", venue_type = "", venue_category = "")

    #### the dataframe to hold the extracted fields, each identified by its MSOA ID.

    dfs["venue_df"] = pd.DataFrame([], columns = ["msoa_id", "venue_name", "venue_location", "venue_category", "venue_type"])

    #### Due to the aforementioned limitation of FourSquare's free tier, raw responses are decieded
    #### to be saved in a serialized format,
    #### should additional data be intended to be extracted.

    raw_responses = []
    data_length = len(dfs["all"].index)
    i = 0
    #### Main loop to iterate through each MSOA center within the dataframe.

    while(i < data_length):
        try:
            msoa_id =  dfs["all"].loc[i, "msoa11cd"]

            #### Getting the raw response data for the location
            raw_response = get_fs_for_latlng(dfs["all"].loc[i, "coordinates"], raw = True)
            raw_responses.append(raw_response)
            venue_ds_temp = venue_ds_template.copy()
            for k,v in venue_extractor_functions.items():
                temp_list =  get_venue_fields(raw_response, v)
                venue_ds_temp[k] = temp_list
            venue_ds_temp["msoa_id"] = [msoa_id for i in range(0, len(temp_list))]
            dfs["venue_df"] = dfs["venue_df"].append(pd.DataFrame(venue_ds_temp))
            print("\rPulled data for {:d}/{:d}. Last MSOID: {:s}".format(i+1, data_length, msoa_id))
            i+=1
        except:
            print("*"*50 + "\nI will go to sleep for 2 hours since have reached the limit of FourSquare's free tier.\n" + "*"*50)
            time.sleep(60*60*2+10)
    return raw_responses

In [ ]:
 #patient_foursquare_calls(dfs)

### Loading the requested FourSquare data

In [ ]:
dfs["foursquare_data"] = pd.read_csv("fsquare_all.csv")

In [ ]:
dfs["foursquare_data"].columns

In [ ]:
dfs["foursquare_data"].rename(columns = {"msoa_id": "msoa11cd"}, inplace = True)

In [ ]:
dfs["foursquare_data"].head(3)

In [ ]:
dfs["foursquare_data"]["venue_category"].unique()

In [ ]:
boolean_mask_asian = dfs["foursquare_data"]["venue_category"].str.match(re.compile(".*(cambodian|korea|japanese|chinese|asian|vietnamese|thai|laos|Taiwan).*(restaurant|pub).*",  re.IGNORECASE))
boolean_mask_asian.sum()

In [ ]:
boolean_mask_hosp = dfs["foursquare_data"]["venue_category"].str.match(re.compile(".*(restaurant|pub|food|takeaway).*",  re.IGNORECASE))
boolean_mask_hosp.sum()

In [ ]:
dfs["foursquare_data"][boolean_mask_asian].head()

In [ ]:
asian_gb = dfs["foursquare_data"][boolean_mask_asian].groupby(by="msoa11cd").count().reset_index()
asian_gb.shape

In [ ]:
dfs["foursquare_data"][boolean_mask_hosp].head()

In [ ]:
hosp_gb = dfs["foursquare_data"][boolean_mask_hosp].groupby(by="msoa11cd").count().reset_index()
hosp_gb.shape

In [ ]:
hosp_gb.head(3)

In [ ]:
aggr_asian_hosp = asian_gb[["msoa11cd", "venue_name"]]\
.rename(columns = {"venue_name": "asian_hospitality_count"})\
.merge(hosp_gb[["msoa11cd", "venue_name"]].rename(columns = {"venue_name": "all_hospitality_count"}),
       how = "left",
       left_on = "msoa11cd",
       right_on = "msoa11cd")

In [ ]:
aggr_asian_hosp["asian_hospitality_percentage"] = aggr_asian_hosp["asian_hospitality_count"] / aggr_asian_hosp["all_hospitality_count"]

In [ ]:
aggr_asian_hosp.head()

### Merging the retrieved FourSquare data with the main DataFrame

In [ ]:
dfs["all"] = dfs["all"].merge(aggr_asian_hosp, how = "left", left_on = "msoa11cd", right_on="msoa11cd")
dfs["all"].head(3)

In [ ]:
dfs["all"].isna().sum()

In [ ]:
dfs["all"].drop(columns = ["all_hospitality_count"], inplace = True)

In [ ]:
dfs["all"] = dfs["all"].merge(hosp_gb[["msoa11cd", "venue_type"]], how = "left")
dfs["all"].rename(columns = {"venue_type": "all_hospitality_count"}, inplace = True)

In [ ]:
dfs["all"].isna().sum()

In [ ]:
dfs["all"].fillna(0, inplace = True)

In [ ]:
dfs["all"].isna().sum()

In [ ]:
dfs["all"]["pop_relative_hospitality_frequency"] = dfs["all"]["all_hospitality_count"] / dfs["all"]["total_population"]

In [ ]:
filtered_list  = filter_geo_json_with_tooltip(json_response, MSOA_codes, dfs_all = dfs["all"], dfs_tooltip_cols = "asian_hospitality_percentage")

### Displaying the relative ratio of asian hospitality facilities by MSOA in a Choloropeth map

In [ ]:
maps["asian_frequency"] = maps["base_map"]()
maps["asian_frequency_choropleth"] = folium.Choropleth(geo_data = json_response , 
                                                       data = dfs["all"], 
                                                       columns = ["msoa11cd", "asian_hospitality_percentage"],
                                                       key_on = "feature.properties.msoa11cd",
                                                       fill_color = "BuPu")
maps["asian_frequency_choropleth"].geojson.add_child(
    folium.features.GeoJsonTooltip(label = False, fields = ["asian_hospitality_percentage"])
)

In [ ]:
maps["asian_frequency_choropleth"].add_to(maps["asian_frequency"])
#maps["asian_frequency"]

<h2 id="Analysing-each-MSOA-by-the-relative-frequency-of-the-restaurants-serving-meals-of-the-selected-cuisine">Analysing each MSOA by the relative frequency of the restaurants serving meals of the selected cuisine w.r.t. the frequency of other restaurants....<br /><br /></h2>

### Analysing clustering tendency

In [ ]:
def get_hopkins_statistics(dataset, sample_percentage = 0.1):

    np.random.seed(0)
    ds_shape = dataset.shape

    sample_size = int(ds_shape[0] * sample_percentage)
    
    artificial_ds = np.random.uniform(np.amin(dataset, axis = 0), np.amax(dataset, axis = 0), size = [sample_size, ds_shape[1]])

    sample_indices = np.random.choice(range(0, ds_shape[0]), sample_size)
    ds_samples = dataset[sample_indices]

    m = NearestNeighbors().fit(dataset)
    
    u_dist, _ = m.kneighbors(artificial_ds, n_neighbors = 2, return_distance = True)
    w_dist, _ = m.kneighbors(ds_samples, n_neighbors = 2, return_distance = True)
    
    # Why n_neighbors = 2?
    # If n_neighbors = 1 is given, instead of a calculated distance as one sum it returns the distance dimensionwise
    
    # For w_dist it is the second neighbor, hence the 1th index. 
    # The first neighbour is the data point itself since it has been sampled from the dataset.
    # For u_dist, it is the first neighbor (hence the 0th index), because
    # the artificial datapoints are assumed to be not part of the original dataset.

    w_dist = w_dist[:,1]
    u_dist = u_dist[:, 0]
    return np.sum(u_dist)/(np.sum(w_dist) + np.sum(u_dist))
    

### Assessing the clustering tendency of the real dataset

In [ ]:
dfs["all"].columns

### Feature selection

In [ ]:
selected_feature_columns = ["total_annual_income",
                            "asian_population_percentage",
                            "asian_hospitality_percentage",
                            "pop_relative_hospitality_frequency",
                            "average_rent"]

In [ ]:
column_data_for_hopkins_statistics = dfs["all"]\
[selected_feature_columns].values
column_data_for_hopkins_statistics = StandardScaler().fit_transform(column_data_for_hopkins_statistics)

In [ ]:
column_data_for_hopkins_statistics.shape

In [ ]:
get_hopkins_statistics(column_data_for_hopkins_statistics)

In [ ]:
hopkins(column_data_for_hopkins_statistics,110)

# Finding the optimal K for clustering

from yellowbrick.cluster import KElbowVisualizer

# Instantiate the clustering model and visualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(3,11))

visualizer.fit(X)
visualizer

In [ ]:
kays = list(range(2, 20))
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
silhouette_scores = []
wss_scores = []

In [ ]:
for i in kays:
    model = KMeans(init = "k-means++", random_state = 0, n_init = 500, n_clusters = i)
    model.fit(column_data_for_hopkins_statistics)
    silhouette_scores.append(silhouette_score(column_data_for_hopkins_statistics, model.labels_, metric = 'euclidean'))
    wss_scores.append(model.inertia_)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
silhouette_scores_hat = MinMaxScaler().fit_transform(np.array(silhouette_scores).reshape(18,1))
wss_scores_hat =  MinMaxScaler().fit_transform(np.array(wss_scores).reshape(18,1))

In [ ]:
fig = plt.figure()
ax = plt.gca()
ax.plot(kays, silhouette_scores_hat, label = "Silhouette scores")
ax.plot(kays, wss_scores_hat, label = "Within Cluster Sum of Squares")
ax.plot([4,4], [0,1.2], label = "Identified ideal K number")
ax.set_xticks(list(range(1,21)))
ax.set_xlabel("Number of K")
ax.legend()

In [ ]:
model = KMeans(init = "k-means++", random_state = 0, n_init = 500, n_clusters = 4)
model.fit(column_data_for_hopkins_statistics)
dfs["all"]["cluster"] = model.labels_

### Identifying the optimal clusters

In [ ]:
selected_feature_columns.append("cluster")
selected_feature_columns

In [ ]:
dfs["all"][selected_feature_columns].groupby(by="cluster").median()

In [ ]:
dfs["all"][selected_feature_columns].groupby(by="cluster").mean()

# Visualise clusters on the map

In [ ]:
filtered_list  = filter_geo_json_with_tooltip(json_response,
                                              MSOA_codes,
                                              dfs_all = dfs["all"], 
                                              dfs_tooltip_cols = selected_feature_columns,
                                                tooltip_name = "comprehensive")
json_response["features"] = filtered_list

In [ ]:
final_map = maps['base_map']()
final_choro = folium.Choropleth(
    data = dfs["all"],
    geo_data = json_response,
    columns = ["msoa11cd", "cluster"],
    key_on = "feature.properties.msoa11cd",
    fill_color = "BuPu")
tooltips = folium.features.GeoJsonTooltip(fields = ["comprehensive"], labels = False)

final_choro.geojson.add_child(
      tooltips  
)
final_map.add_child(final_choro)